In [81]:




import pandas as pd
pd.set_option('display.max_columns',40)
import os
os.chdir(r"C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_profit")

In [82]:
# Path to your data folder in Google Drive
data_folder = r"C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_profit"
# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['processed_100%_ideal_profit.csv']

In [83]:
s = pd.read_csv('processed_100%_ideal_profit.csv')

In [84]:
s.reset_index(drop=True,inplace=True)

In [39]:
# s = pd.read_csv('Processed_5%_RF_5%_results (2).csv')
# s1 = pd.read_csv('Processed_10%_RF_10%_results (2).csv')
# s2 = pd.read_csv('Processed_20%_RF_20%_results (2).csv')
# s3 = pd.read_csv('Processed_50%_RF_50%_results (2).csv')
# s4 = pd.read_csv('Processed_80%_RF_80%_results (2).csv')
# s5 = pd.read_csv('Processed_100%_RF_100%_results (3).csv')

# print(s.duplicated().sum())
# print(s1.duplicated().sum())
# print(s2.duplicated().sum())
# print(s3.duplicated().sum())
# print(s4.duplicated().sum())
# print(s5.duplicated().sum())


In [162]:
s.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit',
       'Actual_ideal_profit'],
      dtype='object')

In [93]:
s.shape

(65462, 27)

In [94]:
s.duplicated().sum()

39

In [52]:
s.drop_duplicates(inplace=True)

In [53]:
s.duplicated().sum()

0

For 100% with less than equal to condition


In [85]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add timestamp columns
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0

    # Add timestamp columns
    df[f'PIT_Profit_Timestamp_{percent}'] = ''
    df[f'PIT_D_Profit_Timestamp_{percent}'] = ''
    df[f'PIT_Loss_Timestamp_{percent}'] = ''

    print(f"Initialized columns for percent {percent}")

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        if i + 1 >= len(df):  # Skip processing for the last row(s) that cannot slice
            break

        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
               # print(f"Buyers In Control Logic Triggered for index {i}")

                # Defence Check
                low_future_rows = df.iloc[i+1:][['Low', 'Time Stamp']]

                if not low_future_rows.empty:
                    matched_low_index = low_future_rows[low_future_rows['Low'] <= defence].index

                    if not matched_low_index.empty:
                        matched_low_index = matched_low_index[0]  # First occurrence
                        max_high = df.loc[i:matched_low_index, 'High'].max()
                        max_high_rows = df.loc[df['High'] == max_high]

                        if not max_high_rows.empty:
                            max_high_timestamp = max_high_rows['Time Stamp'].values[0]
                        else:
                            max_high_timestamp = ''  # Default value if no max_high found

                        max_close = df.loc[i:matched_low_index, 'Close'].max()

                        df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                        df.loc[i, f'PIT_Aggresive_Target_{percent}'] = max_high

                        # Target_Predicted_X_dis vs. Max High Value Comparison
                        if row[target_col] <= max_high:
                            df.loc[i, f'PIT_Profit_{percent}'] = abs(round((row[target_col] - entry), 2))
                            df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = max_high_timestamp
                        else:
                            stop = row['Stop']
                            if stop != 0:
                                df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 0.25

                                matched_stop_index = low_future_rows[low_future_rows['Low'] <= stop].index

                                if not matched_stop_index.empty:
                                    matched_stop_index = matched_stop_index[0]  # First occurrence
                                    max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                    max_high_stop_rows = df.loc[df['High'] <= max_high_stop]

                                    if not max_high_stop_rows.empty:
                                        max_high_stop_timestamp = max_high_stop_rows['Time Stamp'].values[0]
                                    else:
                                        max_high_stop_timestamp = ''  # Default value if no max_high_stop found

                                    # Defence Target vs. Max High Value Comparison
                                    if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
                                        df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                        df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = max_high_stop_timestamp
                                    else:
                                        df.loc[i, f'PIT_Loss_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                        df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = max_high_stop_timestamp

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                #print(f"Sellers In Control Logic Triggered for index {i}")

                # Defence Check
                high_future_rows = df.iloc[i+1:][['High', 'Time Stamp']]

                if not high_future_rows.empty:
                    matched_high_index = high_future_rows[high_future_rows['High'] >= defence].index

                    if not matched_high_index.empty:
                        matched_high_index = matched_high_index[0]  # First occurrence
                        min_low = df.loc[i:matched_high_index, 'Low'].min()
                        min_low_rows = df.loc[df['Low'] == min_low]

                        if not min_low_rows.empty:
                            min_low_timestamp = min_low_rows['Time Stamp'].values[0]
                        else:
                            min_low_timestamp = ''  # Default value if no min_low found

                        min_close = df.loc[i:matched_high_index, 'Close'].min()

                        df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                        df.loc[i, f'PIT_Aggresive_Target_{percent}'] = min_low

                        # Target_Predicted_X_dis vs. Min Low Value Comparison
                        if row[target_col] >= min_low:
                            df.loc[i, f'PIT_Profit_{percent}'] = abs(round((entry - row[target_col]), 2))
                            df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = min_low_timestamp
                        else:
                            stop = row['Stop']
                            if stop != 0:
                                df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 0.25

                                matched_stop_index = high_future_rows[high_future_rows['High'] >= stop].index

                                if not matched_stop_index.empty:
                                    matched_stop_index = matched_stop_index[0]  # First occurrence
                                    min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                    min_low_stop_rows = df.loc[df['Low'] == min_low_stop]

                                    if not min_low_stop_rows.empty:
                                        min_low_stop_timestamp = min_low_stop_rows['Time Stamp'].values[0]
                                    else:
                                        min_low_stop_timestamp = ''  # Default value if no min_low_stop found

                                    # Defence Target vs. Min Low Value Comparison
                                    if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
                                        df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
                                        df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = min_low_stop_timestamp
                                    else:
                                        df.loc[i, f'PIT_Loss_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)
                                        df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = min_low_stop_timestamp

        # Additional Calculations
        df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
        df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']
        

    # Save the updated dataframe to a CSV file
    output_file = r'C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_prediction\trading_output_updated_ZB(1)_abs.csv'  # Define the file path where you want to save the CSV
    df.to_csv(output_file, index=False)
    print(f"Output saved to {output_file}")


In [86]:
d=apply_trading_logic(s,'Predicted_Ideal','100')

Initialized columns for percent 100


  0%|          | 137/54768 [00:00<01:16, 713.43it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_20676\440174971.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '112.31' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
C:\Users\dell\AppData\Local\Temp\ipykernel_20676\440174971.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '113.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Aggresive_Target_{percent}'] = max_high
C:\Users\dell\AppData\Local\Temp\ipykernel_20676\440174971.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.68' has dtype incompatible with int64, please explicitly c

Output saved to C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_prediction\trading_output_updated_ZB(1)_abs.csv


In [87]:
data=pd.read_csv(r'C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_prediction\trading_output_updated_ZB(1)_abs.csv')

C:\Users\dell\AppData\Local\Temp\ipykernel_20676\1279787504.py:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r'C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_prediction\trading_output_updated_ZB(1)_abs.csv')


In [88]:
data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit',
       'Actual_ideal_profit', 'PIT_Profit_100', 'PIT_D_Profit_100',
       'PIT_Loss_100', 'PIT_Target_Buyer_100', 'PIT_Defence_Target_Buyer_100',
       'PIT_Target_Seller_100', 'PIT_Defence_Target_Seller_100',
       'PIT_Defence_Target_100', 'PIT_Aggresive_Target_100',
       'PIT_Profit_Timestamp_100', 'PIT_D_Profit_Timestamp_100',
       'PIT_Loss_Timestamp_100', 'PIT_Ideal_Target_100'],
      dtype='object')

In [89]:
data.drop(columns=['PIT_Aggresive_Target_100','PIT_Target_Buyer_100','PIT_Defence_Target_Buyer_100','PIT_Target_Seller_100','PIT_Defence_Target_Seller_100','PIT_Ideal_Target_100'],inplace=True,axis=1)

In [90]:
data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit',
       'Actual_ideal_profit', 'PIT_Profit_100', 'PIT_D_Profit_100',
       'PIT_Loss_100', 'PIT_Defence_Target_100', 'PIT_Profit_Timestamp_100',
       'PIT_D_Profit_Timestamp_100', 'PIT_Loss_Timestamp_100'],
      dtype='object')

In [91]:
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df):
    # Initialize the new columns with 0
    df['T_Profit'] = 0
    df['D_Profit'] = 0
    df['Loss'] = 0
    df['Ideal_Target_Buyer'] = 0
    df['Defence_Target_Buyer'] = 0
    df['Ideal_Target_Seller'] = 0
    df['Defence_Target_Seller'] = 0

    # Initialize mismatch counters
    buyers_mismatch_count = 0
    sellers_mismatch_count = 0
    buyers_stop_mismatch_count = 0
    sellers_stop_mismatch_count = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check with <= condition
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows <= defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, 'Ideal_Target_Buyer'] = max_close

                    # Target vs. Max High Value Comparison
                    if row['Target'] <= max_high:
                        df.loc[i, 'T_Profit'] = abs(row['Target'] - entry)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
                            matched_stop_index = low_future_rows[low_future_rows <= stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, 'Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, 'D_Profit'] = abs(df.loc[i, 'Defence_Target_Buyer'] - entry)
                                else:
                                    df.loc[i, 'Loss'] = abs(df.loc[i, 'Defence_Target_Buyer'] - entry)
                            else:
                                buyers_stop_mismatch_count += 1
                else:
                    buyers_mismatch_count += 1

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check with >= condition
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows >= defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, 'Ideal_Target_Seller'] = min_close

                    # Target vs. Min Low Value Comparison
                    if row['Target'] >= min_low:
                        df.loc[i, 'T_Profit'] = abs(entry - row['Target'])
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Seller'] = ((entry + defence) / 2) - 0.25
                            matched_stop_index = high_future_rows[high_future_rows >= stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, 'Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, 'D_Profit'] = abs(entry - df.loc[i, 'Defence_Target_Seller'])
                                else:
                                    df.loc[i, 'Loss'] = abs(df.loc[i, 'Defence_Target_Seller'] - entry)
                            else:
                                sellers_stop_mismatch_count += 1
                else:
                    sellers_mismatch_count += 1

    # Print mismatch counts
    print(f"Buyers mismatch count: {buyers_mismatch_count}")
    print(f"Sellers mismatch count: {sellers_mismatch_count}")
    print(f"Buyers stop mismatch count: {buyers_stop_mismatch_count}")
    print(f"Sellers stop mismatch count: {sellers_stop_mismatch_count}")

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Ideal_Target_Buyer", "Ideal_Target_Seller", "Defence_Target_Buyer", "Defence_Target_Seller"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    return df


In [92]:
data = apply_trading_logic(data)

  0%|          | 0/54768 [00:00<?, ?it/s]

C:\Users\dell\AppData\Local\Temp\ipykernel_20676\850206605.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '112.31' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Ideal_Target_Buyer'] = max_close
C:\Users\dell\AppData\Local\Temp\ipykernel_20676\850206605.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '112.55000000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
C:\Users\dell\AppData\Local\Temp\ipykernel_20676\850206605.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2400000000000091' has dtype incompatible with int64, please explicitly cast to a compatible dtyp

Buyers mismatch count: 0
Sellers mismatch count: 0
Buyers stop mismatch count: 0
Sellers stop mismatch count: 0


In [93]:
data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit',
       'Actual_ideal_profit', 'PIT_Profit_100', 'PIT_D_Profit_100',
       'PIT_Loss_100', 'PIT_Defence_Target_100', 'PIT_Profit_Timestamp_100',
       'PIT_D_Profit_Timestamp_100', 'PIT_Loss_Timestamp_100', 'T_Profit',
       'D_Profit', 'Loss'],
      dtype='object')

In [94]:
data.drop(columns=['T_Profit', 'D_Profit'],axis=1,inplace=True)

In [95]:
data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit',
       'Actual_ideal_profit', 'PIT_Profit_100', 'PIT_D_Profit_100',
       'PIT_Loss_100', 'PIT_Defence_Target_100', 'PIT_Profit_Timestamp_100',
       'PIT_D_Profit_Timestamp_100', 'PIT_Loss_Timestamp_100', 'Loss'],
      dtype='object')

In [96]:
data.to_csv(r"C:\Users\dell\Desktop\Updated_ZB\Updated_ZB_dataset1_100%_prediction\Processed_distnce_updated_ideal_target_1_version_abs_updated_ZB.csv",index=False)

**Aggressive**


In [1]:
import pandas as pd
pd.set_option('display.max_columns',40)
import os
os.chdir("C:\\Users\\dell\\Desktop\\Non_imputed\\Non_imputed_100%_profit")

In [2]:
# Path to your data folder in Google Drive
data_folder_1 = "C:\\Users\\dell\\Desktop\\Non_imputed\\Non_imputed_100%_profit"

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder_1)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['Processed_100%_newModeltest_Agressive_data_non_imputed (1).csv']

In [3]:
s = pd.read_csv("Processed_100%_newModeltest_Agressive_data_non_imputed (1).csv")
# s1 = pd.read_csv("Processed_10%_Updated_RF_Agressive_10%_results (1).csv")
# s2 = pd.read_csv("Processed_20%_Updated_RF_Agressive_20%_results (1).csv")
# s3 = pd.read_csv("Processed_50%_Updated_RF_Agressive_50%_results (1).csv")
# s4 = pd.read_csv("Processed_80%_Updated_RF_Agressive_80%_results (1).csv")
# s5 = pd.read_csv("Processed_100%_Updated_RF_Agressive_100%_results (1).csv")

In [4]:
print(s.columns)
# print(s1.columns)
# print(s2.columns)
# print(s3.columns)
# print(s4.columns)
# print(s5.columns)

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit'],
      dtype='object')


In [5]:
# s.columns

In [6]:
s.shape

(85172, 26)

In [7]:
s.duplicated().sum()

22

In [9]:
s.drop_duplicates(inplace=True)

In [10]:
s.duplicated().sum()

0

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 'Aggressive' in the column names
    df[f'PAT_Profit_{percent}'] = 0
    df[f'PAT_D_Profit_{percent}'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Loss'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Target_Buyer'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Target_Seller'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Defence_Target'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Buyer'] = max_close
                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = max_high

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PAT_Profit_{percent}'] = round(abs(row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, f'PAT_D_Profit_{percent}'] = abs(df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] - entry)
                                else:
                                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Loss'] = abs(df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] - entry)

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Seller'] = min_close
                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = min_low

                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PAT_Profit_{percent}'] = round(abs(entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] = ((entry + defence) / 2) - 0.25
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, f'PAT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'])
                                else:
                                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Loss'] = abs(df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] - entry)

        # Additional Calculations
        df.loc[i, f'Target_Predicted_Aggressive_{percent}_Ideal_T'] = df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Buyer'] + df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Seller']
        df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target'] = df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] + df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller']

    return df

# def process_files(input_dir, output_dir):
#     # List all CSV files in the input directory
#     files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

#     for file in files:
#         # Extract the percentage from the filename
#         percent = file.split('_')[1].replace('%', '')

#         # Load the DataFrame
#         df = pd.read_csv(os.path.join(input_dir, file))

#         # Determine the Target_Predicted_X_dis column to use
#         target_col = f'Predicted_Agressive_{percent}_dis'
#         # Target_Predicted_Agressive_100_dis

#         # Apply trading logic
#         df = apply_trading_logic(df, target_col, percent)

#         # Drop unnecessary columns before saving
#         columns_to_drop = [
#             f'Target_Predicted_Aggressive_{percent}_Target_Buyer',
#             f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer',
#             f'Target_Predicted_Aggressive_{percent}_Target_Seller',
#             f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'
#         ]
#         df.drop(columns=columns_to_drop, inplace=True)

#         # Save the processed DataFrame to the output directory
#         output_file = f'Processed_{percent}_{file}'
#         df.to_csv(os.path.join(output_dir, output_file), index=False)

# Example usage

# output_dir = "C:\\Users\\dell\\Desktop\\Non_imputed\\Non_imputed_100%_prediction"
# process_files(data_folder_1, output_dir)


  0%|          | 0/85172 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_16472\1267156648.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4638.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = max_high
  0%|          | 4/85172 [00:00<04:14, 334.62it/s]


KeyError: 'Predicted_Agressive_100_dis'

In [ ]:
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df):
    # Initialize the new columns with 0
    df['T_Profit'] = 0
    df['D_Profit'] = 0
    df['Loss'] = 0
    df['Ideal_Target_Buyer'] = 0
    df['Defence_Target_Buyer'] = 0
    df['Ideal_Target_Seller'] = 0
    df['Defence_Target_Seller'] = 0

    # Initialize mismatch counters
    buyers_mismatch_count = 0
    sellers_mismatch_count = 0
    buyers_stop_mismatch_count = 0
    sellers_stop_mismatch_count = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check with <= condition
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows <= defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, 'Ideal_Target_Buyer'] = max_close

                    # Target vs. Max High Value Comparison
                    if row['Target'] <= max_high:
                        df.loc[i, 'T_Profit'] = abs(row['Target'] - entry)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
                            matched_stop_index = low_future_rows[low_future_rows <= stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, 'Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, 'D_Profit'] = abs(df.loc[i, 'Defence_Target_Buyer'] - entry)
                                else:
                                    df.loc[i, 'Loss'] = abs(df.loc[i, 'Defence_Target_Buyer'] - entry)
                            else:
                                buyers_stop_mismatch_count += 1
                else:
                    buyers_mismatch_count += 1

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check with >= condition
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows >= defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, 'Ideal_Target_Seller'] = min_close

                    # Target vs. Min Low Value Comparison
                    if row['Target'] >= min_low:
                        df.loc[i, 'T_Profit'] = abs(entry - row['Target'])
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Seller'] = ((entry + defence) / 2) - 0.25
                            matched_stop_index = high_future_rows[high_future_rows >= stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, 'Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, 'D_Profit'] = abs(entry - df.loc[i, 'Defence_Target_Seller'])
                                else:
                                    df.loc[i, 'Loss'] = abs(df.loc[i, 'Defence_Target_Seller'] - entry)
                            else:
                                sellers_stop_mismatch_count += 1
                else:
                    sellers_mismatch_count += 1

    # Print mismatch counts
    print(f"Buyers mismatch count: {buyers_mismatch_count}")
    print(f"Sellers mismatch count: {sellers_mismatch_count}")
    print(f"Buyers stop mismatch count: {buyers_stop_mismatch_count}")
    print(f"Sellers stop mismatch count: {sellers_stop_mismatch_count}")

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Ideal_Target_Buyer", "Ideal_Target_Seller", "Defence_Target_Buyer", "Defence_Target_Seller"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    return df
